In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
%cd ..

/cis/home/dpacker/my_documents/dark-matter-halos


In [2]:
from src.subsampling.subsample import *
import numpy as np
from pathlib import Path
import pickle
import gzip
from jax import numpy as jnp
rng = np.random.default_rng(seed=0)

root_path = Path(".")
data_path = root_path / "data"
generated_data_path = root_path / "generated_data_extended"

In [3]:
FILE = 'halo_pointclouds_extended.pkl.gz'
with gzip.open(data_path / FILE, "r") as f:
    points_list, velocities_list, halos_sel = pickle.load(f)   
points_list = [preprocess_pointcloud(p) for p in points_list]


In [4]:
downsample_size = 75
n_trials = 1
sampled_points, sampled_weights = kmeans_downsample_points(
    points_list, downsample_size, n_trials, pbar=True
)

sampled_velocities = np.zeros((sampled_points.shape[0], n_trials, downsample_size, 3))
for i in tqdm(range(sampled_points.shape[0])):
    for t in range(n_trials):
        sampled_velocities[i, t] = aggregate_velocities(
            points_list[i], recenter(velocities_list[i]), sampled_points[i, t]
        )

jnp.savez(
    generated_data_path / f"kmeans_subsampled_n{n_trials}_s{downsample_size}.npz",
    points=sampled_points,
    weights=sampled_weights,
    velocities=sampled_velocities,
)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [10]:
jnp.savez(generated_data_path / "features_and_targets.npz",
        **halos_sel
          )

In [9]:
halos_sel.keys()

dict_keys(['StellarMass', 'HalfMassRad', 'StellarMetallicity', 'StarFormRate', 'SubhaloC200', 'Group_M_Crit200'])